### Read Lexems file


In [ ]:
fs = require('fs')

In [ ]:
lexer_str = fs.readFileSync('symbols.ho').toString()

### Parse the lexems

Create the tokenizer which each regex

In [ ]:
tokenizer = {}
lexems = lexer_str.split(';')
if(lexems[lexems.length - 1] === '') lexems.pop()
lexems.forEach(l => { 
    let parsed = l.split(':=')
    console.log(parsed)
    regex = "^" + parsed[1].trim()
    tokenizer[parsed[0].trim()] = new RegExp(regex)
})

In [ ]:
tokenizer

In [ ]:
function toTokens(phrase){
    let tokens = []
    for(cursor = 0; cursor < phrase.length; ++cursor){
        // ignore spaces
        if(phrase[cursor] === " "){
            continue
        }
        let found = false
        for (let el of Object.entries(tokenizer)){
            let regex = el[1]
            let tokenName = el[0]
            let res = phrase.slice(cursor).match(regex)
            if(res){
                console.log("wow we found a " + tokenName + " with value " + res)
                tokens.push({type: tokenName, value: res[0]})
                found = true
                cursor += res[0].length - 1
                break
            }
        }
        if(!found){
            console.log("arf, could not tokenize character " + phrase[cursor])
        }
    }
    return tokens
}

In [ ]:
input = "(Faisan + Multicolore):1 \\> Etre -> Oiseau, *Chasseurs* -> (Chasser + Beaucoup) -> @1."

In [ ]:
tokens = toTokens(input)

In [ ]:
grammar_str = fs.readFileSync('grammar.ho').toString()

In [ ]:
grammar = {}
rules = grammar_str.split(';')
rules.pop()
rules.forEach(r => { 
    let parsed = r.split(':=')
    let tokens = parsed[1].split('|').map(tok => tok.trim().split(' '))
    grammar[parsed[0].trim()] = tokens
})
grammar

on prend un token
on prend toutes les regles qui commencent par ce token
pour chaque regle, tant qu'elles peuvent consommer des token on continue à prendre des tokens
( mot + mot )

In [ ]:
function parse(tokens, rules){
    console.log("RULES")
    console.log(tokens, rules)
    if(tokens.length === 0){
        console.log("no more tokens")
        return
    }
    for(rule in rules){
        filter_rules = rules[rule].filter(r => r[0] == tokens[0].type)
        if(filter_rules.length > 0){
            console.log(tokens[0], filter_rules)
            newRules = {}
            newRules[rule] = filter_rules
            parse(tokens.slice(1, ), newRules)
        }
    }
}


In [1]:
test_grammar = "texte := ( phrase fin_phrase ) + ;\n phrase := mot id ?;"

'texte := ( phrase fin_phrase ) + ;\n phrase := mot id ?;'